# Preamble

In [1]:
import xarray as xr
import numpy as np
import holoviews as hv
from pathlib import Path
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np
from datetime import date
from matplotlib import pyplot as plt
# matplotlib widget
import dask.array as da
# from dask.distributed import Client, LocalCluster
# # import param
# from holoviews import streams
import csv
# Get working directory
import sys, os
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import marineHeatWaves as mhw
from marineHeatWaves import detect

## Import satellite data

In [2]:
# SET UO OF THE PARAMETERS THE CONTROL THE FILTER DATA

# First filter (buffer): cells close to the land
buffer = 0.8 # buffer size in kilometers

# Second filter (a): cells with strong difference with respect to the neighboring ones in a given time
temperature_threshold_a = 2 # Set your desired temperature difference threshold: decrease it to mask more

# Third filter (b): cells with strong difference with respect to the spatial average in a given time
temperature_threshold_b = 5 # Set your desired temperature difference threshold abs(T-T_spatial_average): decrease it to mask more

In [3]:
# linear interpolated datasets
fn_garda = 'interpolated/ID505-garda-LSWT-19990901_20200901-v2.0.1-linear.interp.nc'

# set full paths
path_lswt_garda = Path(module_path).joinpath('../data/'+fn_garda)

# load lswt dataset as xarray.Dataset
ds_lswt_garda = xr.open_dataset(path_lswt_garda, chunks='auto')

In [4]:
# access data variable using point
da_lswt_garda = ds_lswt_garda.lake_surface_water_temperature

# Slice based on data avilability

# Define time interval
start_date = '2007-01-01'
end_date = '2020-09-01'

# Slice the DataArray based on the time interval
da_lswt_garda = da_lswt_garda.sel(time=slice(start_date, end_date))

In [5]:
# # Convert from Kelvin to Celcius
# da_lswt_garda = da_lswt_garda - 273.15
# da_lswt_garda.attrs['description'] = 'Lake Surface Water Temperature'
# da_lswt_garda.attrs['unit'] = 'degrees Celsius (°C)'

In [6]:
# set-up a buffer to remove cells close to shore using the distance_to_land data variable

da_lswt_buffered_garda = da_lswt_garda.where(ds_lswt_garda['distance_to_land']>buffer)

In [7]:
# Function to calculate the second minimum absolute temperature difference
def calculate_second_min_absolute_difference(da, threshold, high_value=1000):
    # Create an array with surrounding points for each point
    surrounding_points = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0),  (1, 1)
    ]

    # Initialize the minimum and second minimum differences
    min_difference = np.inf
    second_min_difference = np.inf

    # Iterate through each surrounding point and calculate the absolute difference
    for offset_lat, offset_lon in surrounding_points:
        # Shift the data array by the offset
        shifted_da = da.shift(lat=offset_lat, lon=offset_lon)

        # Replace NaN values in the original and shifted arrays with a high value
        da_with_nan_replaced = da.where(~np.isnan(da), high_value)
        shifted_da_with_nan_replaced = shifted_da.where(~np.isnan(shifted_da), high_value)

        # Calculate the absolute difference between the original and shifted arrays
        temp_difference = np.abs(da_with_nan_replaced - shifted_da_with_nan_replaced)

        # Update the minimum and second minimum differences
        second_min_difference = np.minimum(np.maximum(min_difference, temp_difference), second_min_difference)
        min_difference = np.minimum(min_difference, temp_difference)

    # Create a mask based on the second minimum absolute difference criterion
    mask = (second_min_difference <= threshold) | (second_min_difference > high_value / 2)

    return mask


# Calculate the mask
mask_a = calculate_second_min_absolute_difference(da_lswt_buffered_garda, temperature_threshold_a, high_value=1000)
# Apply the mask to the original data array
da_lswt_buffered_masked_garda_a = da_lswt_buffered_garda.where(mask_a)

In [8]:
# Step 1: Calculate the spatial average temperature for each day
spatial_avg_temperature = da_lswt_buffered_masked_garda_a.mean(dim=['lat', 'lon'])
# Step 2: Calculate the absolute difference between spatial average and each cell's temperature
abs_difference_b = np.abs(da_lswt_buffered_masked_garda_a - spatial_avg_temperature)

# Step 4: Create a mask based on the absolute difference criterion
mask_b = abs_difference_b <= temperature_threshold_b
# Step 5: Apply the mask to the previously buffered and masked data array
da_lswt_buffered_masked_garda_b = da_lswt_buffered_masked_garda_a.where(mask_b)

# Satellite data Analisis

In [9]:
da_lswt = da_lswt_buffered_masked_garda_b
da_lswt

<xarray.DataArray 'lake_surface_water_temperature' (time: 4993, lat: 52, lon: 43)>
dask.array<where, shape=(4993, 52, 43), dtype=float32, chunksize=(4993, 52, 43), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2020-09-01
Attributes:
    long_name:             lake surface water temperature
    units:                 degrees celsius
    interpolation_method:  interp_method
    description:           Interpolated lake surface water temperature derive...

In [24]:
plot = da_lswt.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 30),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=4992, width=550)

In [10]:
# Count number of cells to analyze
# Count non-null values along the 'time' dimension
count_temp = da_lswt.count(dim='time')

# Find the cells that have temperature data at least once
cells_with_data = count_temp > 0

# Sum over the 'lat' and 'lon' dimensions
total_cells_with_data = cells_with_data.sum(dim=['lat', 'lon'])

print("Total number of cells containing temperature data over time:", int(total_cells_with_data))

Total number of cells containing temperature data over time: 452


In [11]:
# Extract the lat and lon indices where data are present
lat_lon_pairs = np.column_stack(np.where(cells_with_data))

# Import Modelled Series

In [12]:
# Function to format and import the simulated data
def import_and_format(lat_idx, lon_idx):
    file_path_cc = f'../air2water-master/Garda/output_4/2_PSO_RMS_{lat_idx}_{lon_idx}_cc_1d.out'
    file_path_cv = f'../air2water-master/Garda/output_4/3_PSO_RMS_{lat_idx}_{lon_idx}_cv_1d.out'
    
    # Import the data
    df_cc = pd.read_csv(file_path_cc, sep='\s+', header=None, usecols=[0, 1, 2, 5], names=['Year', 'Month', 'Day', 'Sim_Temp'])
    df_cv = pd.read_csv(file_path_cv, sep='\s+', header=None, usecols=[0, 1, 2, 5], names=['Year', 'Month', 'Day', 'Sim_Temp'])
    
    # Filter out rows where "Year" is equal to -999 (warming up year)
    df_cc = df_cc[df_cc['Year'] != -999]
    df_cv = df_cv[df_cv['Year'] != -999]
    
    # Reset index
    df_cc.reset_index(drop=True, inplace=True)
    df_cv.reset_index(drop=True, inplace=True)
    
    # Concat calibration and validation data
    df = pd.concat([df_cv, df_cc], ignore_index=True)
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    
    # Drop the individual year, month, and day columns
    df.drop(columns=['Year', 'Month', 'Day'], inplace=True)
    
    # Reorder
    df = df[['Date', 'Sim_Temp']]
    
    return df

# Dictionary to hold the formatted DataFrames
simulated_dataframes = {}

# Iterate through each pair of lat and lon indices
for lat_idx, lon_idx in lat_lon_pairs:
    df = import_and_format(lat_idx, lon_idx)
    simulated_dataframes[f"sim_df_{lat_idx}_{lon_idx}"] = df

# Print the DataFrames
for name, df in simulated_dataframes.items():
    print(f"{name}:\n{df.head()}\n")


sim_df_1_21:
        Date  Sim_Temp
0 1990-01-01   8.55958
1 1990-01-02   8.43933
2 1990-01-03   8.33432
3 1990-01-04   8.25638
4 1990-01-05   8.17504

sim_df_2_17:
        Date  Sim_Temp
0 1990-01-01   8.69636
1 1990-01-02   8.59119
2 1990-01-03   8.49679
3 1990-01-04   8.42388
4 1990-01-05   8.34738

sim_df_2_18:
        Date  Sim_Temp
0 1990-01-01   8.51264
1 1990-01-02   8.43630
2 1990-01-03   8.36478
3 1990-01-04   8.30721
4 1990-01-05   8.24559

sim_df_2_19:
        Date  Sim_Temp
0 1990-01-01   8.85182
1 1990-01-02   8.73329
2 1990-01-03   8.62900
3 1990-01-04   8.54979
4 1990-01-05   8.46765

sim_df_2_20:
        Date  Sim_Temp
0 1990-01-01   8.79095
1 1990-01-02   8.69029
2 1990-01-03   8.59906
3 1990-01-04   8.52743
4 1990-01-05   8.45230

sim_df_2_21:
        Date  Sim_Temp
0 1990-01-01   8.55124
1 1990-01-02   8.47168
2 1990-01-03   8.39728
3 1990-01-04   8.33700
4 1990-01-05   8.27293

sim_df_3_6:
        Date  Sim_Temp
0 1990-01-01   8.93060
1 1990-01-02   8.85409
2 1990-

In [13]:
# Take the first DataFrame from the dictionary to initialize the DataArray
first_df = list(simulated_dataframes.values())[0]

# Initialize an empty DataArray with the new dimensions
sim_da_lswt = xr.DataArray(np.nan, coords=[first_df['Date'], da_lswt.lat, da_lswt.lon], dims=['time', 'lat', 'lon'])

# Iterate through the simulated_dataframes dictionary to assign the data
for name, df in simulated_dataframes.items():
    # Extract the latitude and longitude indices from the DataFrame name
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    
    # Assign temperature data to the corresponding position in the DataArray
    sim_da_lswt.loc[dict(lat=da_lswt.lat.values[lat_idx], lon=da_lswt.lon.values[lon_idx])] = df['Sim_Temp'].values


# sim_da_lswt


In [14]:
plot = sim_da_lswt.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 30),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=11322, width=550)

In [15]:
# Function to calculate climatology using the marineHeatWaves detect function
def calculate_climatology(df):
    dates = df['Date'].apply(lambda x: x.toordinal()).values  # Convert dates to ordinal format
    temps = df['Sim_Temp'].values
    
    # Using the detect function to calculate climatology
    mhw_events, clim = mhw.detect(dates, temps)
    
    return clim

# Dictionary to hold the climatology results
climatologies = {}

# Iterate through the DataFrames and calculate the climatology
for name, df in simulated_dataframes.items():
    climatology = calculate_climatology(df)
    climatologies[name] = climatology

# Print the climatologies
for name, clim in climatologies.items():
    print(f"{name} climatology:\n{clim}\n")


sim_df_1_21 climatology:
{'thresh': array([9.67320681, 9.6077081 , 9.54193745, ..., 9.8862939 , 9.81158132,
       9.74133842]), 'seas': array([8.82020042, 8.75677288, 8.69532658, ..., 9.02533777, 8.95421202,
       8.88591576]), 'missing': array([False, False, False, ..., False, False, False])}

sim_df_2_17 climatology:
{'thresh': array([ 9.79672006,  9.72948103,  9.66313523, ..., 10.01552135,
        9.93878232,  9.86645135]), 'seas': array([8.99344729, 8.92837142, 8.86526361, ..., 9.20312544, 9.13058642,
       9.06076565]), 'missing': array([False, False, False, ..., False, False, False])}

sim_df_2_18 climatology:
{'thresh': array([ 9.85946961,  9.79114123,  9.7244951 , ..., 10.08700058,
       10.00812768,  9.93159155]), 'seas': array([9.03452582, 8.96848478, 8.90464778, ..., 9.24797741, 9.17409501,
       9.10299627]), 'missing': array([False, False, False, ..., False, False, False])}

sim_df_2_19 climatology:
{'thresh': array([ 9.8718489 ,  9.80384084,  9.73658697, ..., 10.0937

In [16]:
# Create the threshold dataArray
example_df = list(climatologies.values())[0]  # Get an example climatology DataFrame to get the time dimension
new_time_dim = len(example_df['thresh'])
thresh_values = np.full((new_time_dim, da_lswt.shape[1], da_lswt.shape[2]), np.nan)
seas_values = np.full((new_time_dim, da_lswt.shape[1], da_lswt.shape[2]), np.nan)


# Populate the array with threshold values from the climatologies
for name, clim in climatologies.items():
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    thresh_values[:, lat_idx, lon_idx] = clim['thresh']

# Create the da_thresh DataArray
da_thresh = xr.DataArray(
    thresh_values,
    coords=[sim_da_lswt['time'], da_lswt.lat, da_lswt.lon],
    dims=['time', 'lat', 'lon']
)

# Populate the array with threshold values from the climatologies
for name, clim in climatologies.items():
    lat_idx, lon_idx = map(int, name.split('_')[2:])
    seas_values[:, lat_idx, lon_idx] = clim['seas']

# Create the da_thresh DataArray
da_clim = xr.DataArray(
    seas_values,
    coords=[sim_da_lswt['time'], da_lswt.lat, da_lswt.lon],
    dims=['time', 'lat', 'lon']
)

da_thresh

<xarray.DataArray (time: 11323, lat: 52, lon: 43)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 9.34543403,
         9.30320042,        nan],
        [       nan,        nan,        nan, ..., 9.6036229 ,
         9.35126381, 9.14897784],
        [       nan,        nan,        nan, ..., 9.44176316,
         9.34827177,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ..., 9.47640242,
         9.43020268,        nan],
        [       nan,        nan,        nan, ..., 9.73536258,
         9.48195671, 9.27524913],
        [       nan,        nan,        nan, ..., 9.55995284,
         9.47165306,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ..., 9.41008113,
         9.36548558,        nan],
        [       nan,        nan,        nan, ..., 9.66886452,
         9.415938  , 9.21140429],
        [       nan,        nan,        nan, ..., 9.50028639,
         9.40852823,        nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1990-01-01 1990-01-02 ... 2020-12-31
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86

In [17]:
# Align the time coordinates
common_dates = np.intersect1d(da_lswt.time.values, da_thresh.time.values)

da_lswt_aligned = da_lswt.sel(time=common_dates)
da_thresh_aligned = da_thresh.sel(time=common_dates)
da_clim_aligned = da_clim.sel(time=common_dates)

# Ensure both arrays have the same time dimension
assert len(da_lswt_aligned.time) == len(da_thresh_aligned.time)

In [18]:
# Calculate intensities
da_intensity = da_lswt - da_clim_aligned
da_extrathresh = da_lswt - da_thresh_aligned
da_intensity

# Calculate categories
diff = da_thresh_aligned - da_clim_aligned

# Calculate category thresholds
category1_thresh = da_clim_aligned + 2 * diff
category2_thresh = da_clim_aligned + 3 * diff
category3_thresh = da_clim_aligned + 4 * diff

# Initialize the 'category' variable with NaNs
da_category = xr.full_like(da_lswt, np.nan, dtype=int)

# Assign category 0 where lswt is less than thresh, preserving NaN values
da_category = xr.where(da_lswt < da_thresh_aligned, 0, da_category)

da_category = xr.where(da_lswt > da_thresh_aligned, 1, da_category)
da_category = xr.where((da_lswt > category1_thresh) & (da_lswt <= category2_thresh), 2, da_category)
da_category = xr.where((da_lswt > category2_thresh) & (da_lswt <= category3_thresh), 3, da_category)
da_category = xr.where(da_lswt > category3_thresh, 4, da_category)

# Preserve NaN values from da_lswt
da_category = xr.where(da_lswt.isnull(), np.nan, da_category)

In [19]:
dataset_garda = xr.Dataset({
    'lswt': da_lswt,
    'clim': da_clim_aligned,
    'thresh': da_thresh_aligned,
    'intensity': da_intensity,
    'category': da_category,
    'extrathresh' : da_extrathresh
})
dataset_garda

<xarray.Dataset>
Dimensions:      (lat: 52, lon: 43, time: 4993)
Coordinates:
  * lat          (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.86 45.87
  * lon          (lon) float64 10.51 10.52 10.53 10.54 ... 10.85 10.85 10.86
  * time         (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2020-09-01
Data variables:
    lswt         (time, lat, lon) float32 dask.array<chunksize=(4993, 52, 43), meta=np.ndarray>
    clim         (time, lat, lon) float64 nan nan nan nan ... 21.42 20.94 nan
    thresh       (time, lat, lon) float64 nan nan nan nan ... 22.29 21.52 nan
    intensity    (time, lat, lon) float64 dask.array<chunksize=(4993, 52, 43), meta=np.ndarray>
    category     (time, lat, lon) float64 dask.array<chunksize=(4993, 52, 43), meta=np.ndarray>
    extrathresh  (time, lat, lon) float64 dask.array<chunksize=(4993, 52, 43), meta=np.ndarray>

In [20]:
# Save the Dataset
dataset_garda.to_netcdf('../../output/interpolated_lake_dataset_with_clim.nc')

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


In [26]:
custom_cmap = ['white', 'gold', 'darkorange', 'darkred', 'black']

plot2 = da_category.hvplot(
    groupby='time',
    geo=True,
    cmap= custom_cmap , clim=(-0.5, 4.5),
    xlabel='Longitude', ylabel='Latitude',
    clabel='Category (1:Moderate - 2:Strong - 3:Severe - 4:Extreme)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot2

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=4992, width=550)

In [22]:
da_clim = dataset_garda.clim
da_clim

<xarray.DataArray 'clim' (time: 4993, lat: 52, lon: 43)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,  8.74496192,
          8.5667677 ,         nan],
        [        nan,         nan,         nan, ...,  8.94304424,
          8.70392712,  8.49672947],
        [        nan,         nan,         nan, ...,  8.70967643,
          8.6671961 ,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [        nan,         nan,         nan, ..., 20.97548878,
         20.84776042,         nan],
        [        nan,         nan,         nan, ..., 20.7531366 ,
         20.89856041, 20.88882998],
        [        nan,         nan,         nan, ..., 21.49000048,
         20.99893986,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ..., 20.91888391,
         20.79063124,         nan],
        [        nan,         nan,         nan, ..., 20.70225662,
         20.84285893, 20.82941179],
        [        nan,         nan,         nan, ..., 21.42082355,
         20.93896803,         nan]]])
Coordinates:
  * lat      (lat) float64 45.45 45.45 45.46 45.47 ... 45.85 45.85 45.86 45.87
  * lon      (lon) float64 10.51 10.52 10.53 10.54 ... 10.84 10.85 10.85 10.86
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 ... 2020-09-01

In [32]:
plot = da_clim.hvplot(
    groupby='time',
    geo=True,
    cmap='jet', clim=(5, 20),
    xlabel='Longitude', ylabel='Latitude',
    clabel='LSWT (°C)',
    tiles='CartoLight',
    width=300,
    widget_type='scrubber', widget_location='bottom'
)
plot

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom', widget_type='scrubber')
    [1] WidgetBox(align=('center', 'end'))
        [0] Player(end=4992, width=550)